In [ ]:
# Install the kaggle API
! pip install kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rishabhranka09","key":"fc19d1c0e98c6cefc99fe3e6be9b0a99"}'}

In [ ]:
# 5. Make a directory named kaggle and copy kaggle.json file there
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

# Change the permission of the file
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# 6. Download the Netflix data
!kaggle datasets download -d netflix-inc/netflix-prize-data

Dataset URL: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data
License(s): other
 98% 670M/683M [00:06<00:00, 87.0MB/s]
100% 683M/683M [00:06<00:00, 115MB/s] 


In [ ]:
# 7. Unzip the downloaded file
!unzip netflix-prize-data.zip

Archive:  netflix-prize-data.zip
  inflating: README                  
  inflating: combined_data_1.txt     
  inflating: combined_data_2.txt     
  inflating: combined_data_3.txt     
  inflating: combined_data_4.txt     
  inflating: movie_titles.csv        
  inflating: probe.txt               
  inflating: qualifying.txt          


Load the Data
The code below load the data from combined_data_1.txt as a data frame. Expand the code so to load from all data files.

In [ ]:
!pip install pyspark

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
if not os.path.isfile('data.csv'):
    data = open('data.csv', mode='w')

file = "combined_data_1.txt"
with open(file) as f:
    for line in f:
        line = line.strip()
        if line.endswith(':'):
            movie_id = line.replace(':', '')
        else:
            data.write(movie_id + ',' + line)
            data.write('\n')

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Collaborative Filtering').getOrCreate()

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType
schema = StructType([
    StructField("movie_id", IntegerType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("ratings", IntegerType(), True),
    StructField("data", DateType(), True)
])
data_customer = spark.read.csv('data.csv', header=False, schema=schema)
data_customer.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- ratings: integer (nullable = true)
 |-- data: date (nullable = true)



2. Train-Test Split
Split the dataset into 80% training data and 20% test data.

In [ ]:
from pyspark.ml.tuning import TrainValidationSplit

# Split data into train and test set
train, test = data_customer.randomSplit([0.8, 0.2], seed=42)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="movie_id", ratingCol="ratings",
          coldStartStrategy="drop")
model = als.fit(train)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="ratings",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

Root-mean-square error = 0.9562315522187478


In [ ]:
userRecs.head(5)

[Row(user_id=126, recommendations=[Row(movie_id=382, rating=7.5892791748046875), Row(movie_id=1517, rating=7.558063983917236), Row(movie_id=4294, rating=7.448196887969971), Row(movie_id=3328, rating=6.913433074951172), Row(movie_id=2894, rating=6.816694259643555), Row(movie_id=4186, rating=6.7647223472595215), Row(movie_id=970, rating=6.757270812988281), Row(movie_id=51, rating=6.726584434509277), Row(movie_id=3502, rating=6.467933654785156), Row(movie_id=2215, rating=6.419711112976074)]),
 Row(user_id=133, recommendations=[Row(movie_id=4076, rating=5.2866621017456055), Row(movie_id=4300, rating=4.22236442565918), Row(movie_id=396, rating=4.082207679748535), Row(movie_id=3504, rating=3.9577057361602783), Row(movie_id=4471, rating=3.80117130279541), Row(movie_id=2311, rating=3.7570817470550537), Row(movie_id=1729, rating=3.681417465209961), Row(movie_id=876, rating=3.666825294494629), Row(movie_id=2593, rating=3.6604843139648438), Row(movie_id=4158, rating=3.603008508682251)]),
 Row(use

In [ ]:
movieRecs.head(5)

[Row(movie_id=1, recommendations=[Row(user_id=1798925, rating=8.32288646697998), Row(user_id=2532574, rating=8.256898880004883), Row(user_id=588589, rating=7.991403102874756), Row(user_id=2396335, rating=7.856922149658203), Row(user_id=1431451, rating=7.712429046630859), Row(user_id=1616550, rating=7.537178993225098), Row(user_id=2486222, rating=7.509316921234131), Row(user_id=1917856, rating=7.486286640167236), Row(user_id=2595431, rating=7.455434799194336), Row(user_id=1715195, rating=7.4374260902404785)]),
 Row(movie_id=3, recommendations=[Row(user_id=1258170, rating=6.866022109985352), Row(user_id=263902, rating=6.754932403564453), Row(user_id=2634372, rating=6.6894707679748535), Row(user_id=1510892, rating=6.677548885345459), Row(user_id=1106934, rating=6.623396396636963), Row(user_id=706841, rating=6.482214450836182), Row(user_id=1088015, rating=6.444871425628662), Row(user_id=383247, rating=6.429018974304199), Row(user_id=234746, rating=6.416256427764893), Row(user_id=1367254, r